In [1]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adadelta, Adagrad, Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras import backend as K
from keras.losses import mean_squared_error
import tensorflow as tf

# Normalize the data.
from sklearn import preprocessing
from keras.regularizers import l1_l2

import random

def pass_arg(nsim, tr_size):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # MC dropout
    class MCDropout(Dropout):
        def call(self, inputs, training=None):
            return super(MCDropout, self).call(inputs, training=True)


    # import pickle

    # def save_obj(obj, name):
    #     with open(name, 'wb') as f:
    #         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    
    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

    # Making sure dimensionless bond length is less than 1
    def bond(bl):
        return tf.add(K.relu(tf.negative(bl)), K.relu(bl-1.0))

    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        return K.relu(tf.negative(porof)) + K.relu(porof-poroi)

    def strength1(bl, porof, nlayer=6):
        sigma01, sigma02, C1s = 6, 31, 21
        sigma_long = sigma01*(K.exp((1.0-porof)**(C1s*nlayer))-porof) + sigma02*(1.0-porof)
        n = K.shape(sigma_long)[0]  
        sorted_strength, sortedIndices = tf.math.top_k(sigma_long, n, True)
        sorted_bl = K.gather(bl, sortedIndices)
        sorted_porof = K.gather(porof, sortedIndices)
        argg = tf.argsort(sorted_bl,axis=-1,direction='DESCENDING',stable=False,name=None)
        sorted_bl_corr = K.gather(sorted_bl, argg)
        return sorted_bl_corr-sorted_bl


    def strength2(bl, porof, nlayer=6):
        sigma01, sigma02, C1s = 6, 31, 21
        sigma_long = sigma01*(K.exp((1.0-porof)**(C1s*nlayer))-porof) + sigma02*(1.0-porof)
        n = K.shape(sigma_long)[0]  
        sorted_strength, sortedIndices = tf.math.top_k(sigma_long, n, True)
        sorted_bl = K.gather(bl, sortedIndices)
        n = K.cast(n, tf.float32)
        rel = K.relu(sorted_bl[1:]-sorted_bl[0:-1])
        num_vio = K.cast(tf.math.count_nonzero(rel), tf.float32)
        return num_vio/n


    def phy_loss_mean(params):
        # useful for cross-checking training
        loss1, loss2, loss3, loss4, lam1, lam2, lam3, lam4 = params
        def loss(y_true,y_pred):
    #         return lam1*K.mean(K.relu(loss1)) + lam2*K.mean(K.relu(loss2)) + lam2*K.mean(K.relu(loss3))
            return lam1*K.mean(K.relu(loss1)) + lam2*K.mean(K.relu(loss2)) + lam3*K.mean(K.relu(loss3)) + lam4*loss4
        return loss

    #function to calculate the combined loss = sum of rmse and phy based loss
    def combined_loss(params):
        loss1, loss2, loss3, loss4, lam1, lam2, lam3, lam4 = params
        def loss(y_true,y_pred):
    #         return mean_squared_error(y_true, y_pred) + lam1 * K.mean(K.relu(loss1)) + lam2 * K.mean(K.relu(loss2)) + lam2 * K.mean(K.relu(loss3))
            return mean_squared_error(y_true, y_pred) + lam1 * K.mean(K.relu(loss1)) + lam2 * K.mean(K.relu(loss2)) + lam3*K.mean(K.relu(loss3)) + lam4 * loss4
        return loss

    def PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, iteration, n_layers, n_nodes, tr_size, lamda, reg, samp):

    #     fix_seeds(ss)

        # Hyper-parameters of the training process
    #     batch_size = tr_size
        batch_size = 1
        num_epochs = 50
        val_frac = 0.2
        patience_val = 50

        # Initializing results filename
        exp_name = optimizer_name + '_drop' + str(drop_frac) + '_usePhy' + str(use_YPhy) +  '_nL' + str(n_layers) + '_nN' + str(n_nodes) + '_trsize' + str(tr_size) + '_lamda' + str(lamda) + '_iter' + str(iteration)
        exp_name = exp_name.replace('.','pt')
        results_dir = '../results/'
        model_name = results_dir + exp_name + '_model.h5' # storing the trained model

        if reg==True and samp==25:
            results_name = results_dir + exp_name + '_results_25_regularizer.dat' # storing the results of the model
        elif reg==False and samp==25:
            results_name = results_dir + exp_name + '_results_25.dat' # storing the results of the model
        elif reg==True and samp==1519:
            results_name = results_dir + exp_name + '_results_1519_regularizer.dat' # storing the results of the model
        elif reg==False and samp==1519:
            results_name = results_dir + exp_name + '_results_1519.dat' # storing the results of the model

        # Load labeled data
        data = np.loadtxt('../data/labeled_data.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_unique.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_v2.dat')
#         x_labeled = data[:, :2] # -2 because we do not need porosity predictions
        x_label = data[:, :-3] # -2 because we do not need porosity predictions
        x_labeled = np.hstack((x_label[:,:2],x_label[:,-2:]))
        y_labeled = data[:, -3:-1] # dimensionless bond length and porosity measurements
        if samp==25:
            data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_25.dat')
            x_unlabeled = data[:, :]
        elif samp==1519:
            data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
            x_unlabeled = data[:, :]

        x_unlabeled1 = x_unlabeled[:1303, :]
        x_unlabeled2 = x_unlabeled[-6:, :]
        x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

        # initial porosity
        init_poro = x_unlabeled[:, -1]
        x_unlabeled = np.hstack((x_unlabeled[:,:2],x_unlabeled[:,-3:-1]))
#         x_unlabeled = x_unlabeled[:, :2]


        # normalize dataset with MinMaxScaler
        scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    #     scaler = preprocessing.StandardScaler()
        x_labeled = scaler.fit_transform(x_labeled)
#         y_labeled = scaler.fit_transform(y_labeled)
        x_unlabeled = scaler.fit_transform(x_unlabeled)

    #     # initial porosity & physics outputs are removed
    #     x_unlabeled = x_unlabeled[:, :-3]

        # train and test data
        trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    #     testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]   
        testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]

        if use_YPhy == 0:
            # Removing the last column from x_unlabeled (corresponding to Y_PHY)
            x_unlabeled = x_unlabeled[:,:-1]

        # Creating the model
        model = Sequential()
        for layer in np.arange(n_layers):
            if layer == 0:
                model.add(Dense(n_nodes, activation='relu', input_shape=(np.shape(trainX)[1],)))
            else:
                if reg:
                    model.add(Dense(n_nodes, activation='relu', kernel_regularizer=l1_l2(l1=.001, l2=.001)))
                else:
                    model.add(Dense(n_nodes, activation='relu'))
            # model.add(Dropout(rate=drop_frac))
            model.add(MCDropout(rate=drop_frac))
        model.add(Dense(2, activation='linear'))

        # physics-based regularization
        uinp_sc = K.constant(value=x_unlabeled) # unlabeled input data
        lam1 = K.constant(value=lamda[0]) # regularization hyper-parameter
        lam2 = K.constant(value=lamda[1]) # regularization hyper-parameter
        lam3 = K.constant(value=lamda[2]) # regularization hyper-parameter
        lam4 = K.constant(value=lamda[3]) # regularization hyper-parameter
        predictions = model(uinp_sc) # model output at depth i
    #     porosity = K.relu(predictions[:,1])
        phyloss1 = bond(predictions[:,0]) # physics loss 1
    #     uinp = K.constant(value=x_unlabeled_non) # unlabeled input data
        phyloss2 = poros(init_poro, predictions[:,1]) # physics loss 1
        phyloss3 = strength1(predictions[:,0], predictions[:,1])
        phyloss4 = strength2(predictions[:,0], predictions[:,1])
        totloss = combined_loss([phyloss1, phyloss2, phyloss3, phyloss4, lam1, lam2, lam3, lam4])
        phyloss = phy_loss_mean([phyloss1, phyloss2, phyloss3, phyloss4, lam1, lam2, lam3, lam4])


        model.compile(loss=totloss,
                      optimizer=optimizer_val,
                      metrics=[phyloss, root_mean_squared_error])

        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_val, verbose=1)

    #     print('Running...' + optimizer_name)
        history = model.fit(trainX, trainY,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            verbose=0,
                            validation_split=val_frac, callbacks=[early_stopping, TerminateOnNaN()])
    
#     early_stopping = EarlyStopping(monitor='loss', patience=patience_val, verbose=1)
#     history = model.fit(trainX, trainY,
#                         batch_size=batch_size,
#                         epochs=num_epochs,
#                         verbose=1,
#                         callbacks=[early_stopping, TerminateOnNaN()])

#     test_score = model.evaluate(testX, testY, verbose=0)
#     predictions = model.predict(x_labeled) # model output at depth i
#     print(np.sort(predictions[:,0], axis=0))
    
#     predictions = model.predict(x_unlabeled) # model output at depth i
#     print(np.sort(predictions[:,0], axis=0))
    
#     print('iter: ' + str(iteration) + ' useYPhy: ' + str(use_YPhy) + 
#           ' nL: ' + str(n_layers) + ' nN: ' + str(n_nodes) + 
#           ' lamda1: ' + str(lamda[0]) + ' lamda2: ' + str(lamda[1]) + ' trsize: ' + str(tr_size) + 
#           ' TestRMSE: ' + str(test_score[2]) + ' PhyLoss: ' + str(test_score[1]), ' TestLoss: ' + str(test_score[0]), "\n")

# #     print('iter: ' + str(iteration) + ' TestRMSE: ' + str(test_score[2]) + ' PhyLoss: ' + str(test_score[1]), "\n")

    
# #     model.save(model_name)
    
#     # save results
#     results = {'train_loss_1':history.history['loss_1'], 
#                                 'val_loss_1':history.history['val_loss_1'], 
#                                 'train_rmse':history.history['root_mean_squared_error'], 
#                                 'val_rmse':history.history['val_root_mean_squared_error'],
#                                 'test_rmse':test_score[2],
#                                 'PhyLoss':test_score[1]}

#     results = {'train_loss_1':history.history['loss_1'], 
#                                 'train_rmse':history.history['root_mean_squared_error'], 
#                                 'test_rmse':test_score[2],
#                                 'PhyLoss':test_score[1]}

    

#     save_obj(results, results_name)

#     predictions = model.predict(testX)
#     return results, results_name, predictions, testY, test_score[2], trainY
    
        test_score = model.evaluate(testX, testY, verbose=1)
        print(test_score)

        Xx = np.random.uniform(0,1,(1000,2))
        xx1 = np.ones((1000,2))
        Xx = np.hstack((Xx,xx1))
        
        samples = []
        for i in range(int(nsim)):
#             print("simulation num:",i)
            predictions = model.predict(Xx)
            predictions = predictions[:,1]
            samples.append(predictions)
        return np.array(samples)
    
    



    # Main Function
    if __name__ == '__main__':

        fix_seeds(1)

        # List of optimizers to choose from    
        optimizer_names = ['Adagrad', 'Adadelta', 'Adam', 'Nadam', 'RMSprop', 'SGD', 'NSGD']
        optimizer_vals = [Adagrad(clipnorm=1), Adadelta(clipnorm=1), Adam(clipnorm=1), Nadam(clipnorm=1), RMSprop(clipnorm=1), SGD(clipnorm=1.), SGD(clipnorm=1, nesterov=True)]

        # selecting the optimizer
        optimizer_num = 1
        optimizer_name = optimizer_names[optimizer_num]
        optimizer_val = optimizer_vals[optimizer_num]

        # Selecting Other Hyper-parameters
        drop_frac = 0.1 # Fraction of nodes to be dropped out
        use_YPhy = 1 # Whether YPhy is used as another feature in the NN model or not
        n_layers = 2 # Number of hidden layers
        n_nodes = 5 # Number of nodes per hidden layer

        #set lamda
        lamda = [0.3, 0.15, 0.008, 0] # Physics-based regularization constant  

#         # Iterating over different training fractions and splitting indices for train-test splits
#         trsize_range = [4,6,8,10,20]

#         #default training size = 5000
#         tr_size = trsize_range[4]
        
        tr_size = int(tr_size)

        # use regularizer
        reg = True

        # sample size used
        samp = 1519
    #     samp = 25

        # total number of runs
        iter_range = np.arange(1)
        testrmse=[]
        # iterating through all possible params
        for iteration in iter_range:
#             results, result_file, pred, obs, rmse, obs_train = PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, 
#                             iteration, n_layers, n_nodes, tr_size, lamda, reg, samp)
#             testrmse.append(rmse)
            pred = PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, 
                            iteration, n_layers, n_nodes, tr_size, lamda, reg, samp)
            

    return np.squeeze(pred)

Using TensorFlow backend.


In [2]:
pred = pass_arg(50, 20)

Tr_size: 20
19/19 [==============================] - 0s 53us/step
[0.015868138521909714, 0.000551834877114743, 0.07156015932559967]


In [3]:
mc_pred=np.mean(pred,axis=0)
mc_pred

array([ 2.42983960e-02,  3.10231782e-02,  2.30593011e-02,  1.03699237e-01,
        8.02772492e-02, -2.38511078e-02,  1.52287617e-01,  1.21249348e-01,
        1.91829316e-02,  3.51462997e-02,  7.02235997e-02,  1.63126737e-01,
        1.46971390e-01,  9.72257406e-02,  1.69197172e-01,  3.29563878e-02,
        1.38348728e-01,  1.90577835e-01,  1.35779217e-01,  1.71404623e-04,
        7.47894496e-02,  1.11002102e-01, -2.93192570e-03,  1.06759921e-01,
        3.57108265e-02,  2.97774319e-02,  3.12048346e-02,  1.86630443e-01,
        1.67250514e-01,  1.40376166e-02,  1.29431576e-01,  6.60574213e-02,
        7.16345087e-02,  1.29814506e-01,  1.00107357e-01,  1.27915051e-02,
        1.28592879e-01,  1.12155497e-01,  4.72722277e-02,  6.87448010e-02,
        8.85853078e-03,  3.83499749e-02,  1.94043834e-02, -4.28455975e-03,
        3.69736925e-02,  1.12075433e-01,  1.92998368e-02,  1.50864804e-02,
        8.40060934e-02,  5.52906469e-03,  2.21362650e-01,  1.33668199e-01,
        1.10932849e-01,  

In [4]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mc_pred, "../pred_loss_hyb_MC_Xx.dat")